In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

plt.style.use('ggplot')
%matplotlib inline

In [20]:
ALLEN_AI_RES_PATH = "../res/allenai/metadata.csv"
ALLEN_AI_SUBS_PATH = "../res/allenai/metadata_subset.csv"

In [3]:
df_allen = pd.read_csv(ALLEN_AI_RES_PATH)[['source_x', 'title', 'abstract', 'publish_time', 'journal']]
df_allen['is_pr'] = ~df_allen.source_x.isin(['biorxiv', 'medrxiv', 'arxiv']) # those sources are considered non-peer reviewed

print(len(df_allen.index))
df_allen.head(5)

59887


E:\Python\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,source_x,title,abstract,publish_time,journal,is_pr
0,PMC,Sequence requirements for RNA strand transfer ...,Nidovirus subgenomic mRNAs contain a leader se...,2001-12-17,The EMBO Journal,True
1,PMC,"Crystal structure of murine sCEACAM1a[1,4]: a ...",CEACAM1 is a member of the carcinoembryonic an...,2002-05-01,The EMBO Journal,True
2,PMC,Synthesis of a novel hepatitis C virus protein...,Hepatitis C virus (HCV) is an important human ...,2001-07-16,EMBO J,True
3,PMC,Structure of coronavirus main proteinase revea...,The key enzyme in coronavirus polyprotein proc...,2002-07-01,The EMBO Journal,True
4,PMC,Discontinuous and non-discontinuous subgenomic...,"Arteri-, corona-, toro- and roniviruses are ev...",2002-12-01,The EMBO Journal,True


In [4]:
print('Removing', df_allen.title.isna().sum(), 'entries without titles')
df_allen = df_allen[~df_allen.title.isna()]
print('Remaining', len(df_allen.index))

Removing 163 entries without titles
Remaining 59724


In [5]:
print('Removing', df_allen.title.str.contains('Author index').sum(), 'entries with title of format <x>Author index<z>')
df_allen = df_allen[~df_allen.title.str.contains('Author index')]
print('Remaining', len(df_allen.index))

Removing 37 entries with title of format <x>Author index<z>
Remaining 59687


In [6]:
print('Removing', df_allen[df_allen.title == 'Authors'].count()[0], 'entries with title = "Authors"')
df_allen = df_allen[df_allen.title != 'Authors']
print('Remaining', len(df_allen.index))

Removing 4 entries with title = "Authors"
Remaining 59683


In [7]:
print('Removing', df_allen[df_allen.title == 'Index of Authors'].count()[0], 'entries with title = "Index of Authors"')
df_allen = df_allen[df_allen.title != 'Index of Authors']
print('Remaining', len(df_allen.index))

Removing 0 entries with title = "Index of Authors"
Remaining 59683


In [8]:
print('Removing', df_allen[df_allen.title == 'Information for Authors'].count()[0], 'entries with title = "Information for Authors"')
df_allen = df_allen[df_allen.title != 'Information for Authors']
print('Remaining', len(df_allen.index))

Removing 0 entries with title = "Information for Authors"
Remaining 59683


In [9]:
print('Removing', df_allen[df_allen.title == 'Instructions for Authors'].count()[0], 'entries with title = "Instructions for Authors"')
df_allen = df_allen[df_allen.title != 'Instructions for Authors']
print('Remaining', len(df_allen.index))

Removing 4 entries with title = "Instructions for Authors"
Remaining 59679


In [10]:
print('Removing', df_allen[df_allen.title == 'Authors’ reply'].count()[0], 'entries with title = "Authors’ reply"')
df_allen = df_allen[df_allen.title != 'Authors’ reply']
print('Remaining', len(df_allen.index))

Removing 1 entries with title = "Authors’ reply"
Remaining 59678


In [11]:
print('Removing', df_allen.title.str.contains('Author Index').sum(), 'entries with title of format <x>Author Index<z>')
df_allen = df_allen[~df_allen.title.str.contains('Author Index')]
print('Remaining', len(df_allen.index))

Removing 13 entries with title of format <x>Author Index<z>
Remaining 59665


In [12]:
print('Removing', df_allen.title.str.contains('subject index').sum(), 'entries with title of format <x>subject index<z>')
df_allen = df_allen[~df_allen.title.str.contains('subject index')]
print('Remaining', len(df_allen.index))

Removing 26 entries with title of format <x>subject index<z>
Remaining 59639


In [13]:
print('Removing', df_allen.title.str.contains('Cumulative').sum()-2, 'entries with title of format <x>Cumulative<z>')
df_allen = df_allen[(~df_allen.title.str.contains('Cumulative')) | (df_allen.source_x == 'medrxiv')]
print('Remaining', len(df_allen.index))

Removing 45 entries with title of format <x>Cumulative<z>
Remaining 59595


In [15]:
print('Removing', df_allen[df_allen.title == 'Index'].count()[0], 'entries with title = "Index"')
df_allen = df_allen[df_allen.title != 'Index']
print('Remaining', len(df_allen.index))

Removing 0 entries with title = "Index"
Remaining 59595


In [16]:
print('Removing', df_allen.title.str.contains('Subject Index').sum(), 'entries with title of format <x>Subject Index<z>')
df_allen = df_allen[~df_allen.title.str.contains('Subject Index')]
print('Remaining', len(df_allen.index))

Removing 17 entries with title of format <x>Subject Index<z>
Remaining 59578


In [17]:
print('Removing', df_allen.title.str.contains('Index to').sum(), 'entries with title of format <x>Index to<z>')
df_allen = df_allen[~df_allen.title.str.contains('Index to')]
print('Remaining', len(df_allen.index))

Removing 25 entries with title of format <x>Index to<z>
Remaining 59553


In [18]:
mask = (df_allen.title.str.contains('Index')) & (df_allen.source_x=='Elsevier') & (df_allen.abstract.fillna('Unknown') == 'Unknown')
print('Removing', df_allen[mask].count()[0], 'entries with title of format <x>Index<z>, source=Elsevier and no abstract')
df_allen = df_allen[~mask]
print('Remaining', len(df_allen.index))

Removing 37 entries with title of format <x>Index<z>, source=Elsevier and no abstract
Remaining 59516


In [26]:
print('Removing', df_allen.title.str.contains('Subject index').sum(), 'entries with title of format <x>Subject index<z>')
df_allen = df_allen[~df_allen.title.str.contains('Subject index')]
print('Remaining', len(df_allen.index))

Removing 67 entries with title of format <x>Subject index<z>
Remaining 59449


In [36]:
print('Removing', df_allen.title.str.startswith('Chapter ').sum(), 'entries starting with "Chapter "')
df_allen = df_allen[~df_allen.title.str.startswith('Chapter ')]
print('Remaining', len(df_allen.index))

Removing 1442 entries starting with "Chapter "
Remaining 58007


In [37]:
df_allen[df_allen.title.str.startswith('Chapter')].title.values

array([], dtype=object)

In [38]:
df_allen.to_csv(ALLEN_AI_SUBS_PATH, index=False)

In [21]:
df_elsevier = df_allen[df_allen.source_x=='Elsevier']
starts_with_abstract = len(df_elsevier[df_elsevier.abstract.fillna('').str.startswith('Abstract')].index)
abstract_is_na = df_elsevier.abstract.isna().sum()
print("Starting with 'Abstract'", starts_with_abstract)
print("No Abstract", abstract_is_na)
print("Rest", len(df_elsevier.index) - starts_with_abstract - abstract_is_na)

Starting with 'Abstract' 10860
No Abstract 6999
Rest 4544


In [22]:
len(df_elsevier.index)

22403